<a href="https://colab.research.google.com/github/lalitha-sahitya/Twitter-Sentiment-Analysis/blob/main/twitter_sentiment_analysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [278]:
import os

In [279]:
!pip install opendatasets

In [280]:
import opendatasets as od

In [281]:
od.download('https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis')

Skipping, found downloaded files in "./twitter-entity-sentiment-analysis" (use force=True to force download)


In [282]:
os.listdir('twitter-entity-sentiment-analysis')

['twitter_training.csv', 'twitter_validation.csv']

In [283]:
import pandas as pd
import numpy as np

In [284]:
data=pd.read_csv('/content/twitter-entity-sentiment-analysis/twitter_training.csv',header=None)
data.head(10)

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
5,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
6,2402,Borderlands,Positive,So I spent a few hours making something for fu...
7,2402,Borderlands,Positive,So I spent a couple of hours doing something f...
8,2402,Borderlands,Positive,So I spent a few hours doing something for fun...
9,2402,Borderlands,Positive,So I spent a few hours making something for fu...


In [285]:
data.columns=['id','location','tweet','sentiment']
data.head()

,id,location,tweet,sentiment
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [286]:
val_data=pd.read_csv('/content/twitter-entity-sentiment-analysis/twitter_validation.csv',header=None)
val_data.head()

,0,1,2,3
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [287]:
val_data.columns=['id','location','tweet','sentiment']
val_data.head()

,id,location,tweet,sentiment
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [288]:
data=data.drop(['id', 'location'], axis=1)
val_data=val_data.drop(['id', 'location'], axis=1)

In [289]:
data.head()

,tweet,sentiment
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [290]:
data.isna().sum()

,0
tweet,0
sentiment,686


In [291]:
val_data.isna().sum()

,0
tweet,0
sentiment,0


In [292]:
data=data.dropna()
val_data=val_data.dropna()

In [293]:
data.isna().sum()

,0
tweet,0
sentiment,0


In [294]:
data.duplicated().sum()

4227

In [295]:
val_data.duplicated().sum()

1

In [296]:
data=data.drop_duplicates()
val_data=val_data.drop_duplicates()

In [297]:
data.head()

,tweet,sentiment
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [298]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69769 entries, 0 to 74681
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      69769 non-null  object
 1   sentiment  69769 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


In [299]:

val_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      999 non-null    object
 1   sentiment  999 non-null    object
dtypes: object(2)
memory usage: 23.4+ KB


In [300]:
data['tweet'].value_counts()

,count
tweet,
Negative,21237
Positive,19138
Neutral,17110
Irrelevant,12284


In [301]:
sentences=data['sentiment'].values
labels=data['tweet'].values

In [302]:
val_sentences=val_data['sentiment'].values
val_labels=val_data['tweet'].values

In [303]:
sentences[0]

'im getting on borderlands and i will murder you all ,'

In [304]:
labels[0]

'Positive'

In [305]:
labels[104]

'Negative'

In [306]:
data['tweet'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [315]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
le = LabelEncoder()
labels_int = le.fit_transform(labels)
val_labels_int = le.transform(val_labels)
label = to_categorical(labels_int, num_classes=4)
val_label = to_categorical(val_labels_int, num_classes=4)


In [316]:
!pip install gensim

In [317]:
import re
import gensim
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [318]:
def depure(data):
  url_pattern=re.compile(r'https?://\S+|www\.\S+')
  data=url_pattern.sub(r'',data)
  email=re.compile(r'\S+@\S+')
  data=email.sub(r'',data)
  data=re.sub(r'\s+',' ',data)
  data=re.sub(' ','',data)
  return data

In [319]:
def sent_to_words(sentences):
  for i in sentences:
    yield(gensim.utils.simple_preprocess(str(i),deacc=True))

In [320]:
def detokenize(data):
  return TreebankWordDetokenizer().detokenize(data)

In [321]:
temp=[]
sentences=list(sentences)
for i in range(len(sentences)):
  temp.append(depure(sentences[i]))
words=list(sent_to_words(temp))

In [322]:
d=[]
for i in range(len(words)):
  d.append(detokenize(words[i]))

In [323]:
t=[]
val_sentences=list(val_sentences)
for i in range(len(val_sentences)):
  t.append(depure(val_sentences[i]))
val_words=list(sent_to_words(t))

In [324]:
d1=[]
for i in range(len(val_words)):
  d1.append(detokenize(val_words[i]))

In [325]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [326]:
max_words=5000
max_len=200

In [327]:
tokenizer=Tokenizer(num_words=max_words,oov_token="<OOV>")
tokenizer.fit_on_texts(d)

In [328]:
seq=tokenizer.texts_to_sequences(d)

In [329]:
sequences=pad_sequences(seq,maxlen=max_len)

In [330]:
val_seq=tokenizer.texts_to_sequences(d1,)

In [331]:
val_sequences=pad_sequences(val_seq,maxlen=max_len)

In [332]:
from tensorflow.keras.models import Sequential

In [333]:
label[:20]

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [334]:
model=Sequential([
    keras.layers.Embedding(input_dim=max_words, output_dim=20),
    keras.layers.LSTM(15, dropout=0.5),
    keras.layers.Dense(4, activation='softmax')
])

In [335]:
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [336]:
sequences = np.array(sequences)
val_sequences = np.array(val_sequences)
label = np.array(label)
val_label = np.array(val_label)

In [337]:
sequences.shape

(69769, 200)

In [338]:
len(label)

69769

In [339]:
val_label.shape

(999, 4)

In [340]:
val_sequences.shape

(999, 200)

In [341]:
history=model.fit(sequences, label, epochs=10, validation_data=(val_sequences, val_label))

Epoch 1/10
2181/2181 ━━━━━━━━━━━━━━━━━━━━ 118s 52ms/step - accuracy: 0.3539 - loss: 1.3322 - val_accuracy: 0.4314 - val_loss: 1.2548
Epoch 2/10
2181/2181 ━━━━━━━━━━━━━━━━━━━━ 110s 50ms/step - accuracy: 0.4247 - loss: 1.2362 - val_accuracy: 0.4745 - val_loss: 1.1961
Epoch 3/10
2181/2181 ━━━━━━━━━━━━━━━━━━━━ 135s 47ms/step - accuracy: 0.4646 - loss: 1.1798 - val_accuracy: 0.4935 - val_loss: 1.1427
Epoch 4/10
2181/2181 ━━━━━━━━━━━━━━━━━━━━ 146s 49ms/step - accuracy: 0.4904 - loss: 1.1363 - val_accuracy: 0.4945 - val_loss: 1.1080
Epoch 5/10
2181/2181 ━━━━━━━━━━━━━━━━━━━━ 138s 47ms/step - accuracy: 0.5087 - loss: 1.0999 - val_accuracy: 0.5315 - val_loss: 1.0774
Epoch 6/10
2181/2181 ━━━━━━━━━━━━━━━━━━━━ 142s 47ms/step - accuracy: 0.5197 - loss: 1.0724 - val_accuracy: 0.5225 - val_loss: 1.0730
Epoch 7/10
2181/2181 ━━━━━━━━━━━━━━━━━━━━ 141s 47ms/step - accuracy: 0.5261 - loss: 1.0628 - val_accuracy: 0.5275 - val_loss: 1.0687
Epoch 8/10
2181/2181 ━━━━━━━━━━━━━━━━━━━━ 143s 47ms/step - accuracy: 